The goal here is to prepare 50 queries to be used with Galois.

In [1]:
cd ../..

c:\Users\moham\Desktop\LLM Query\Last Exp\galois


In [2]:
import pandas as pd
from src.utils import get_parsable_queries

In [3]:
df = pd.read_csv('data/Spider_Filtered_Data.csv')

In [4]:
# remove groupby queries
df = df[~df.Query.str.contains('GROUP BY')]

In [5]:
df = df[df.apply(lambda x: get_parsable_queries(x),axis=1)]

In [ ]:
#df.to_csv('Spider_Filtered_Data - Queries_2.csv',index=False)

We then go manually thorugh the file and (i) remove queries that are not generic, such as those containing Singer_ID, and (ii) add query type as a column indicating wether it contains a select operation (S), a projection (P), a where condition (W), an aggregation (A), a nested query (C), and a join (J).

In [6]:
df = pd.read_csv('data/Spider_Filtered_Data_generic.csv')

In [7]:
# remove nested queries
df = df[~df.Type.str.contains('C')]

In [8]:
df['Type2']=df.Type.apply(lambda x: "".join(list(set(x))))

In [9]:
df.groupby(['Database','Type2']).size()

Database  Type2
flight_2  APS       3
          WPAS      1
          WPS       4
flight_4  WPAS      2
          WPS       4
geo       APS       8
          ASPJW     1
          PS        7
          WPAS     14
          WPJS      3
          WPS      25
singer    WPS       1
world_1   ASPJW     4
          WPAS     11
          WPJS      3
          WPS       6
dtype: int64

In [10]:
# split joins an non-joins in order to get 40 non-joins and 10 joins
df_non_join = df[~df.Type2.str.contains('J')]
df_join = df[df.Type2.str.contains('J')]

In [11]:
sampled_join = df_join.sample(10,random_state=42)

In [12]:
df_non_join.Database.value_counts()

Database
geo         54
world_1     17
flight_2     8
flight_4     6
singer       1
Name: count, dtype: int64

In [13]:
sampled_non_join = df_non_join.sample(40,random_state=42)

In [14]:
final_sample_df = pd.concat([sampled_non_join,sampled_join])

We ended up removing 4 more queries that were not compatible (containing not operation, difficult to compute). We end up with 46 queries.

In [ ]:
#final_sample_df.to_csv('data/Final_Queries.csv',index=False)